In [ ]:
from collections import Counter
sorted([(name, Counter(name[0].lower().translate(dict.fromkeys(map(ord, 'aeiounrst .-'), None)))) for name in Card.objects.values_list('name')], key=lambda x: max(x[1].values() or [0]), reverse=True)


In [ ]:
from collections import Counter
Counter(''.join([name[0].lower() for name in Card.objects.values_list('name')]))

In [ ]:
from cards.models import *
print(f'Cards: {Card.objects.count()}')

In [ ]:
print(f'CardPrintings: {CardPrinting.objects.count()}')

In [ ]:
print(f'CardPrintingLanguages: {CardPrintingLanguage.objects.count()}')

In [ ]:
print(f'PhysicalCards: {PhysicalCard.objects.count()}')

In [ ]:
print(f'UserOwnedCards: {UserOwnedCard.objects.filter(owner=User.objects.get(username="Liam")).count()}')

In [ ]:
from django.db.models import Sum
print(f"My Cards: {UserOwnedCard.objects.filter(owner=User.objects.get(username='Liam')).aggregate(Sum('count'))}")

In [ ]:
from django.db import connection
with connection.cursor() as cursor:
    cursor.execute("""
SELECT SUM(reltuples) rowcount
FROM pg_class C
LEFT JOIN pg_namespace N ON (N.oid = C.relnamespace)
WHERE nspname NOT IN ('pg_catalog', 'information_schema') 
AND relkind='r' 
""")
    result = cursor.fetchone()
    
print(result)